In [2]:
!pip install opencv-python matplotlib

ERROR: Could not find a version that satisfies the requirement tira_utils (from versions: none)
ERROR: No matching distribution found for tira_utils


In [2]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from tira_utils import get_input_directory_and_output_directory, normalize_run
import pyterrier as pt
import pandas as pd
import os
import json
from tqdm import tqdm
from glob import glob
from pathlib import Path

Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True


PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
def brute_force_sift_matching(image1, image2):
    # Initiate SIFT detector
    sift = cv.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(image1, None)
    kp2, des2 = sift.detectAndCompute(image2, None)

    # BFMatcher with default params
    bf = cv.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # Apply ratio test
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append([m])

    # cv.drawMatchesKnn expects list of lists as matches.
    img3 = cv.drawMatchesKnn(image1, kp1, image2, kp2, good, None,
                             flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    plt.imshow(img3), plt.show()

def flann_index_kdtree_matcher(image1, image2):
    # Initiate SIFT detector
    sift = cv.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(image1, None)
    kp2, des2 = sift.detectAndCompute(image2, None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)  # or pass empty dictionary

    flann = cv.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1, des2, k=2)

    # Need to draw only good matches, so create a mask
    matchesMask = [[0, 0] for i in range(len(matches))]

    # ratio test as per Lowe's paper
    for i, (m, n) in enumerate(matches):
        if m.distance < 0.7 * n.distance:
            matchesMask[i] = [1, 0]

    draw_params = dict(matchColor=(0, 255, 0),
                       singlePointColor=(255, 0, 0),
                       matchesMask=matchesMask,
                       flags=cv.DrawMatchesFlags_DEFAULT)

    img3 = cv.drawMatchesKnn(image1, kp1, image2, kp2, matches, None,
                             **draw_params)

    plt.imshow(img3, ), plt.show()

def flann_index_lsh_matcher(image1, image2):
    # Initiate SIFT detector
    sift = cv.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(image1, None)
    kp2, des2 = sift.detectAndCompute(image2, None)

    # FLANN parameters
    FLANN_INDEX_LSH = 6
    index_params = dict(algorithm=FLANN_INDEX_LSH,
                        table_number=6,  # 12
                        key_size=12,  # 20
                        multi_probe_level=1)  # 2
    search_params = dict(checks=50)  # or pass empty dictionary

    flann = cv.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1, des2, k=2)

    # Need to draw only good matches, so create a mask
    matchesMask = [[0, 0] for i in range(len(matches))]

    # ratio test as per Lowe's paper
    for i, (m, n) in enumerate(matches):
        if m.distance < 0.7 * n.distance:
            matchesMask[i] = [1, 0]

    draw_params = dict(matchColor=(0, 255, 0),
                       singlePointColor=(255, 0, 0),
                       matchesMask=matchesMask,
                       flags=cv.DrawMatchesFlags_DEFAULT)

    img3 = cv.drawMatchesKnn(image1, kp1, image2, kp2, matches, None,
                             **draw_params)

    plt.imshow(img3, ), plt.show()

In [4]:
import glob

x = glob.glob('dataset22/images/I0*/*/image.webp')

In [5]:
x

['dataset22/images/I00/I000330ba4ea0ad13/image.webp',
 'dataset22/images/I00/I000d8de9c4746ee9/image.webp',
 'dataset22/images/I00/I0010d5b5473065fa/image.webp',
 'dataset22/images/I00/I001652ec040f07c4/image.webp',
 'dataset22/images/I00/I001a318b3176d501/image.webp',
 'dataset22/images/I00/I001b3068979b8629/image.webp',
 'dataset22/images/I00/I001c2de5ef28f77f/image.webp',
 'dataset22/images/I00/I001d567f0fb443d7/image.webp',
 'dataset22/images/I00/I00246a6916fec768/image.webp',
 'dataset22/images/I00/I0024c0097c425855/image.webp',
 'dataset22/images/I00/I002747d3611b43da/image.webp',
 'dataset22/images/I00/I0027bcd7cd6938d9/image.webp',
 'dataset22/images/I00/I00290a8d0fe34013/image.webp',
 'dataset22/images/I00/I002959a3418da38a/image.webp',
 'dataset22/images/I00/I002d16483c9a4e26/image.webp',
 'dataset22/images/I00/I002dd0599859cb69/image.webp',
 'dataset22/images/I00/I002e942703b3e786/image.webp',
 'dataset22/images/I00/I003811155228eb5d/image.webp',
 'dataset22/images/I00/I003a

In [1]:
rootdir = "dataset22/images/I00/*/image.webp"
images = []

for subdir, dirs, files in os.walk(rootdir):
    for directory in subdir:
        os.fsdecode(directory)
        print(directory)

"""for file1 in os.listdir(path):
    for file2 in os.listdir(file1):
        filename = os.fsdecode(file)
        print(file3)"""

"""
for file3 in os.listdir(file2):
            filename = os.fsdecode(file)
            print(file3)
"""  

NameError: name 'os' is not defined

In [17]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        """if filename.endswith(".jpg"):
            img = cv2.imread(os.path.join(folder, filename))
            if img is not None:
                images.append(img)"""
        
        
    
    # return images

In [9]:
SYSTEM_NAME = os.environ.get('TIRA_SYSTEM_NAME' ,'my-retrieval-system')

if not pt.started():
    # tira_utils above should already have done started pyterrier with this configuration to ensure that no internet connection is required (for reproducibility)
    pt.init(version=os.environ['PYTERRIER_VERSION'], helper_version=os.environ['PYTERRIER_HELPER_VERSION'], no_download=True)

input_directory, output_directory = get_input_directory_and_output_directory(default_input='/dataset22')

I will use a small hardcoded example located in /dataset22.
The output directory is /tmp/


In [10]:
print('Step 2: Load the queries.')

def load_queries():
    file_name = input_directory + '/queries.jsonl'
    
    if not os.path.exists(file_name):
        raise ValueError(f'Could not find the file "{file_name}". Got: {glob(input_data + "/*")}')
    
    ret = pd.read_json(file_name, lines=True)

    # https://github.com/terrier-org/pyterrier/issues/62\n",
    ret['query'] = ret['query'].apply(lambda i: "".join([x if x.isalnum() else " " for x in i]))
    
    return ret

queries = load_queries()  
queries.head(2)

Step 2: Load the queries.


NameError: name 'input_data' is not defined

In [11]:
print('Step 3: Create the Index.')

# We use some very baseline method to get a textual representation: we just use the text of the pages that contain the image.
def load_image_text(image_id):
    ret = ''
    
    for txt_file in glob(input_directory +'/images/' + image_id[:3] + '/' + image_id + '/*/*/*/text.txt'):
        ret += '\n\n' + open(txt_file).read()
        
    return ret.strip()

def all_images():
    for i in glob(input_directory + '/images/*/*'):
        image_id = i.split('/')[-1]
        yield {'docno': image_id, 'text': load_image_text(image_id)}


!rm -Rf ./index
iter_indexer = pt.IterDictIndexer("./index", meta={'docno': 20, 'text': 4096})
index_ref = iter_indexer.index(tqdm(all_images()))

Step 3: Create the Index.


0it [00:00, ?it/s]


In [16]:
load_image_text("101")

''

In [15]:
for doc in dataset.get_corpus_iter():
    print(dict(doc))
    break



clueweb12/touche-2022-task-2 documents:   0%|                                                | 0/868655 [00:00<?, ?it/s]

{'text': 'Shuga: Love, Sex, Money MTV Shuga Home Swag Blog Cast Swag Video Team Shuga Partners Shuga Talks (NEW!) Unicef G-PANGE MTV Base MTV Staying Alive Shuga Premiere: Today Is The Day! Today is the day! #ShugaPremiere. We talk Twitter hashtags, competition winners and of course the FREE d/l of the official Shuga: Love, Sex, Money track feat Banky W, WizKid, L-Tido and Bon’eye Download The Official Shuga: Love, Sex, Money Track HERE For Free! Be the first to download the Shuga: Love, Sex, Money track featuring Banky W, WizKid, L-Tido and Bon’eye exclusively here! That Shuga Love Sex Money Premiere The Shuga Track, worldwide Twitter trending and how YOU can win a ticket to the OFFICIAL Shuga: Love, Sex, Money premiere… The Official Shuga: Love, Sex, Money Trailer The Official Shuga: Love, Sex, Money trailer is here!', 'chatnoir_url': 'https://chatnoir.eu/cache?uuid=f338e91e-a3e9-52ec-bb4c-cac853f4df55&index=cw12&raw&plain', 'docno': 'clueweb12-0000tw-14-21168___1'}
